In [1]:
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
import time
import re
import random
from unidecode import unidecode
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [3]:
def GET_UA():
    uastrings = ["Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1500.72 Safari/537.36",\
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10) AppleWebKit/600.1.25 (KHTML, like Gecko) Version/8.0 Safari/600.1.25",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",\
                "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/600.1.17 (KHTML, like Gecko) Version/7.1 Safari/537.85.10",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko",\
                "Mozilla/5.0 (Windows NT 6.3; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.104 Safari/537.36"\
                ]
 
    return random.choice(uastrings)

In [4]:
def parse_url(url):
 
    headers = {'User-Agent': GET_UA()}
    content = None
 
    try:
        response = requests.get(url, headers=headers)
        ct = response.headers['Content-Type'].lower().strip()
 
        if 'text/html' in ct:
            content = response.content
            soup = BeautifulSoup(content)
        else:
            content = response.content
            soup = None
 
    except Exception as e:
        print('Error:', str(e))
 
    return content, soup, ct

In [5]:
codes = pd.read_csv('bballref.txt')

In [6]:
codes['Player'] = codes['Player'].str.replace('*', '', regex=True)
codes['Player'] = codes['Player'].str.replace('\'', '', regex=True)
codes['Player'] = codes['Player'].apply(lambda x: unidecode(x))
codes


,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS▼,Player-additional
0,1,Shaquille ONeal,C,27,LAL,79,79,3163,956,1665,...,336,742,1078,299,36,239,223,255,2344,onealsh01
1,2,Vince Carter,SF,23,TOR,82,82,3126,788,1696,...,150,326,476,322,110,92,178,263,2107,cartevi01
2,3,Karl Malone,PF,36,UTA,82,82,2947,752,1476,...,169,610,779,304,79,71,231,229,2095,malonka01
3,4,Allen Iverson,SG,24,PHI,70,70,2853,729,1733,...,71,196,267,328,144,5,230,162,1989,iversal01
4,5,Gary Payton,PG,31,SEA,82,82,3425,747,1666,...,100,429,529,732,153,18,224,178,1982,paytoga01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14384,535,Thaddeus Young,PF,34,TOR,54,9,795,108,198,...,71,95,166,75,54,5,42,88,240,youngth01
14385,536,Trae Young,PG,24,ATL,73,73,2541,597,1390,...,56,161,217,741,80,9,300,104,1914,youngtr01
14386,537,Omer Yurtseven,C,24,MIA,9,0,83,16,27,...,8,15,23,2,2,2,4,16,40,yurtsom01
14387,538,Cody Zeller,C,30,MIA,15,2,217,37,59,...,25,39,64,10,3,4,14,33,98,zelleco01


In [7]:
codes_dict = pd.Series(codes['Player-additional'].values,index=codes['Player'].values).to_dict()
codes_dict

{'Shaquille ONeal': 'onealsh01',
 'Vince Carter': 'cartevi01',
 'Karl Malone': 'malonka01',
 'Allen Iverson': 'iversal01',
 'Gary Payton': 'paytoga01',
 'Jerry Stackhouse': 'stackje01',
 'Grant Hill': 'hillgr01',
 'Kevin Garnett': 'garneke01',
 'Michael Finley': 'finlemi01',
 'Chris Webber': 'webbech01',
 'Ray Allen': 'allenra02',
 'Alonzo Mourning': 'mournal01',
 'Tim Duncan': 'duncati01',
 'Glenn Robinson': 'robingl01',
 'Antoine Walker': 'walkean02',
 'Shareef Abdur-Rahim': 'abdursh01',
 'Stephon Marbury': 'marbust01',
 'Elton Brand': 'brandel01',
 'Allan Houston': 'houstal01',
 'Antonio McDyess': 'mcdyean01',
 'Keith Van Horn': 'vanhoke01',
 'Latrell Sprewell': 'sprewla01',
 'Sam Cassell': 'cassesa01',
 'Michael Dickerson': 'dickemi01',
 'Kobe Bryant': 'bryanko01',
 'Clifford Robinson': 'robincl02',
 'Reggie Miller': 'millere01',
 'Shawn Kemp': 'kempsh01',
 'Jalen Rose': 'roseja01',
 'Eddie Jones': 'jonesed02',
 'Dirk Nowitzki': 'nowitdi01',
 'Paul Pierce': 'piercpa01',
 'David Rob

In [8]:
index = pd.DataFrame()
years = range(2000,2024)

for y in years:
    year = y
    leaders_url = f'https://www.basketball-reference.com/leagues/NBA_{year}_totals.html#totals_stats::pts'
    print(leaders_url)
    html = urlopen(leaders_url)
    time.sleep(2.95)
    soup = BeautifulSoup(html)
    # use findALL() to get the column headers
    soup.findAll('tr', limit=40)
    # use getText()to extract the text we need into a list
    i = 0
    while i < 40:
        headers = [th.getText() for th in soup.findAll('tr', limit=40)[i].findAll('th')]
        i = i +1
        if headers:
            idx = i
            i = 41

    headers = headers[1:]

    rows = soup.findAll('tr')
    player_stats = [[td.getText() for td in rows[i].findAll('td')]
                for i in range(len(rows))]
    stats = pd.DataFrame(player_stats, columns = headers)
    df = pd.DataFrame()
    df['Player'] = stats.Player
    df['Year'] = y
    index = pd.concat((index,df))
index



https://www.basketball-reference.com/leagues/NBA_2000_totals.html#totals_stats::pts
https://www.basketball-reference.com/leagues/NBA_2001_totals.html#totals_stats::pts
https://www.basketball-reference.com/leagues/NBA_2002_totals.html#totals_stats::pts
https://www.basketball-reference.com/leagues/NBA_2003_totals.html#totals_stats::pts
https://www.basketball-reference.com/leagues/NBA_2004_totals.html#totals_stats::pts
https://www.basketball-reference.com/leagues/NBA_2005_totals.html#totals_stats::pts
https://www.basketball-reference.com/leagues/NBA_2006_totals.html#totals_stats::pts
https://www.basketball-reference.com/leagues/NBA_2007_totals.html#totals_stats::pts
https://www.basketball-reference.com/leagues/NBA_2008_totals.html#totals_stats::pts
https://www.basketball-reference.com/leagues/NBA_2009_totals.html#totals_stats::pts
https://www.basketball-reference.com/leagues/NBA_2010_totals.html#totals_stats::pts
https://www.basketball-reference.com/leagues/NBA_2011_totals.html#totals_sta

,Player,Year
0,None,2000
1,Tariq Abdul-Wahad,2000
2,Tariq Abdul-Wahad,2000
3,Tariq Abdul-Wahad,2000
4,Shareef Abdur-Rahim,2000
...,...,...
701,Thaddeus Young,2023
702,Trae Young,2023
703,Omer Yurtseven,2023
704,Cody Zeller,2023


In [9]:
print(len(index))
index = index.drop_duplicates()
print(len(index))
index= index.dropna()
print(len(index))
index

15010
11546
11522


,Player,Year
1,Tariq Abdul-Wahad,2000
4,Shareef Abdur-Rahim,2000
5,Cory Alexander,2000
6,Ray Allen*,2000
7,Rafer Alston,2000
...,...,...
701,Thaddeus Young,2023
702,Trae Young,2023
703,Omer Yurtseven,2023
704,Cody Zeller,2023


In [10]:
index['Player'] = index['Player'].apply(lambda x: unidecode(x))
index['Player'] = index['Player'].str.replace('*', '', regex=True)
index['Player'] = index['Player'].str.replace('\'', '', regex=True)

In [11]:
index = index.reset_index(drop=True)

In [12]:
index.to_csv('player_index',index=False)

In [13]:
codes_dict[index.Player[292]]

'onealje01'

In [14]:
len(index)

11522

In [40]:
def get_stats(num, start=0):
    failed = []
    for n in range(start,num):
        # NBA season we will be analyzing
        year = index.Year[n]
        player = index.Player[n]
        player_code = codes_dict[player]
        base = f'https://www.basketball-reference.com/players/c/{player_code}/gamelog/'
        time.sleep(2.95)
        # URL page we will scraping (see image above)
        url = base+str(year)
        print(url)

        # this is the HTML from the given URL
        content, soup, ct = parse_url(url)
        
        # use findALL() to get the column headers
        soup.findAll('tr', limit=40)
        # use getText()to extract the text we need into a list
        i = 0
        try:
            while i < 40:
                headers = [th.getText() for th in soup.findAll('tr', limit=40)[i].findAll('th')]
                i = i +1
                if headers:
                    idx = i
                    i = 41

            headers = headers[1:]

            rows = soup.findAll('tr')
            player_stats = [[td.getText() for td in rows[i].findAll('td')]
                        for i in range(len(rows))]
            stats = pd.DataFrame(player_stats, columns = headers)
            stats.dropna(inplace=True)
            stats.insert(0, 'Player', player)
            stats.index = range(len(stats))
            if n == start:
                data_core = stats
            else:
                data_core = pd.concat([data_core,stats])
            print(f'{n}/{num}',year,player)
            if n % 1000 == 0:
                data_core.to_csv('lines\historical',index=False)
        except Exception as error:
            failed.append((year,player))
            print(f"Error: {type(error).__name__}")
            print(error,f'{n}/{num}',year,player)
    return data_core


In [47]:
test = get_stats(len(index),0)

https://www.basketball-reference.com/players/c/abdulta01/gamelog/2000
0/11522 2000 Tariq Abdul-Wahad
https://www.basketball-reference.com/players/c/abdursh01/gamelog/2000
1/11522 2000 Shareef Abdur-Rahim
https://www.basketball-reference.com/players/c/alexaco01/gamelog/2000
2/11522 2000 Cory Alexander
https://www.basketball-reference.com/players/c/allenra02/gamelog/2000
3/11522 2000 Ray Allen
https://www.basketball-reference.com/players/c/alstora01/gamelog/2000
4/11522 2000 Rafer Alston
https://www.basketball-reference.com/players/c/amaecjo01/gamelog/2000
5/11522 2000 John Amaechi
https://www.basketball-reference.com/players/c/anderde01/gamelog/2000
6/11522 2000 Derek Anderson
https://www.basketball-reference.com/players/c/anderke01/gamelog/2000
7/11522 2000 Kenny Anderson
https://www.basketball-reference.com/players/c/anderni01/gamelog/2000
8/11522 2000 Nick Anderson
https://www.basketball-reference.com/players/c/andersh01/gamelog/2000
9/11522 2000 Shandon Anderson
https://www.basketba

83/11522 2000 Bimbo Coles
https://www.basketball-reference.com/players/c/conloma01/gamelog/2000
84/11522 2000 Marty Conlon
https://www.basketball-reference.com/players/c/corbity01/gamelog/2000
85/11522 2000 Tyrone Corbin
https://www.basketball-reference.com/players/c/crawfch01/gamelog/2000
86/11522 2000 Chris Crawford
https://www.basketball-reference.com/players/c/croshau01/gamelog/2000
87/11522 2000 Austin Croshere
https://www.basketball-reference.com/players/c/crottjo01/gamelog/2000
88/11522 2000 John Crotty
https://www.basketball-reference.com/players/c/cummite01/gamelog/2000
89/11522 2000 Terry Cummings
https://www.basketball-reference.com/players/c/cummivo01/gamelog/2000
90/11522 2000 Vonteego Cummings
https://www.basketball-reference.com/players/c/curlebi01/gamelog/2000
91/11522 2000 Bill Curley
https://www.basketball-reference.com/players/c/curryde01/gamelog/2000
92/11522 2000 Dell Curry
https://www.basketball-reference.com/players/c/currymi01/gamelog/2000
93/11522 2000 Michael 

https://www.basketball-reference.com/players/c/harvean01/gamelog/2000
167/11522 2000 Antonio Harvey
https://www.basketball-reference.com/players/c/hawkihe01/gamelog/2000
168/11522 2000 Hersey Hawkins
https://www.basketball-reference.com/players/c/hawkimi01/gamelog/2000
169/11522 2000 Michael Hawkins
https://www.basketball-reference.com/players/c/hendeal01/gamelog/2000
170/11522 2000 Alan Henderson
https://www.basketball-reference.com/players/c/hendece02/gamelog/2000
171/11522 2000 Cedric Henderson
https://www.basketball-reference.com/players/c/hendrma01/gamelog/2000
172/11522 2000 Mark Hendrickson
https://www.basketball-reference.com/players/c/herrech01/gamelog/2000
173/11522 2000 Chris Herren
https://www.basketball-reference.com/players/c/hillgr01/gamelog/2000
174/11522 2000 Grant Hill
https://www.basketball-reference.com/players/c/hillty01/gamelog/2000
175/11522 2000 Tyrone Hill
https://www.basketball-reference.com/players/c/hoibefr01/gamelog/2000
176/11522 2000 Fred Hoiberg
https://

250/11522 2000 Darrick Martin
https://www.basketball-reference.com/players/c/mashbja01/gamelog/2000
251/11522 2000 Jamal Mashburn
https://www.basketball-reference.com/players/c/masonan01/gamelog/2000
252/11522 2000 Anthony Mason
https://www.basketball-reference.com/players/c/masseto01/gamelog/2000
253/11522 2000 Tony Massenburg
https://www.basketball-reference.com/players/c/maxweve01/gamelog/2000
254/11522 2000 Vernon Maxwell
https://www.basketball-reference.com/players/c/mccarwa01/gamelog/2000
255/11522 2000 Walter McCarty
https://www.basketball-reference.com/players/c/mccloge01/gamelog/2000
256/11522 2000 George McCloud
https://www.basketball-reference.com/players/c/mccoyje01/gamelog/2000
257/11522 2000 Jelani McCoy
https://www.basketball-reference.com/players/c/mcdyean01/gamelog/2000
258/11522 2000 Antonio McDyess
https://www.basketball-reference.com/players/c/mcgratr01/gamelog/2000
259/11522 2000 Tracy McGrady
https://www.basketball-reference.com/players/c/mcilvji01/gamelog/2000
26

333/11522 2000 Don Reid
https://www.basketball-reference.com/players/c/reidjr01/gamelog/2000
334/11522 2000 J.R. Reid
https://www.basketball-reference.com/players/c/rhodero01/gamelog/2000
335/11522 2000 Rodrick Rhodes
https://www.basketball-reference.com/players/c/ricegl01/gamelog/2000
336/11522 2000 Glen Rice
https://www.basketball-reference.com/players/c/richmmi01/gamelog/2000
337/11522 2000 Mitch Richmond
https://www.basketball-reference.com/players/c/rideris01/gamelog/2000
338/11522 2000 Isaiah Rider
https://www.basketball-reference.com/players/c/roberst01/gamelog/2000
339/11522 2000 Stanley Roberts
https://www.basketball-reference.com/players/c/roberry01/gamelog/2000
340/11522 2000 Ryan Robertson
https://www.basketball-reference.com/players/c/robincl02/gamelog/2000
341/11522 2000 Clifford Robinson
https://www.basketball-reference.com/players/c/robinda01/gamelog/2000
342/11522 2000 David Robinson
https://www.basketball-reference.com/players/c/robined01/gamelog/2000
343/11522 2000 E

...

2972/11522 2006 Tony Parker
https://www.basketball-reference.com/players/c/patteru01/gamelog/2006
2973/11522 2006 Ruben Patterson
https://www.basketball-reference.com/players/c/paulch01/gamelog/2006
2974/11522 2006 Chris Paul
https://www.basketball-reference.com/players/c/pavloal01/gamelog/2006
2975/11522 2006 Sasha Pavlovic
https://www.basketball-reference.com/players/c/paytoga01/gamelog/2006
2976/11522 2006 Gary Payton
https://www.basketball-reference.com/players/c/perkike01/gamelog/2006
2977/11522 2006 Kendrick Perkins
https://www.basketball-reference.com/players/c/petermo01/gamelog/2006
2978/11522 2006 Morris Peterson
https://www.basketball-reference.com/players/c/petrojo01/gamelog/2006
2979/11522 2006 Johan Petro
https://www.basketball-reference.com/players/c/piatker01/gamelog/2006
2980/11522 2006 Eric Piatkowski
https://www.basketball-reference.com/players/c/piercpa01/gamelog/2006
2981/11522 2006 Paul Pierce
https://www.basketball-reference.com/players/c/pietrmi01/gamelog/2006
29

3054/11522 2006 Kenny Thomas
https://www.basketball-reference.com/players/c/thomaku01/gamelog/2006
3055/11522 2006 Kurt Thomas
https://www.basketball-reference.com/players/c/thomati01/gamelog/2006
3056/11522 2006 Tim Thomas
https://www.basketball-reference.com/players/c/thompdi01/gamelog/2006
3057/11522 2006 Dijon Thompson
https://www.basketball-reference.com/players/c/tinslja01/gamelog/2006
3058/11522 2006 Jamaal Tinsley
https://www.basketball-reference.com/players/c/tsakaja01/gamelog/2006
3059/11522 2006 Jake Tsakalidis
https://www.basketball-reference.com/players/c/tskitni01/gamelog/2006
3060/11522 2006 Nikoloz Tskitishvili
https://www.basketball-reference.com/players/c/turiaro01/gamelog/2006
3061/11522 2006 Ronny Turiaf
https://www.basketball-reference.com/players/c/turkohe01/gamelog/2006
3062/11522 2006 Hedo Turkoglu
https://www.basketball-reference.com/players/c/udokaim01/gamelog/2006
3063/11522 2006 Ime Udoka
https://www.basketball-reference.com/players/c/udrihbe01/gamelog/2006


3136/11522 2007 J.J. Barea
https://www.basketball-reference.com/players/c/bargnan01/gamelog/2007
3137/11522 2007 Andrea Bargnani
https://www.basketball-reference.com/players/c/barnema02/gamelog/2007
3138/11522 2007 Matt Barnes
https://www.basketball-reference.com/players/c/barrean01/gamelog/2007
3139/11522 2007 Andre Barrett
https://www.basketball-reference.com/players/c/barroea01/gamelog/2007
3140/11522 2007 Earl Barron
https://www.basketball-reference.com/players/c/barrybr01/gamelog/2007
3141/11522 2007 Brent Barry
https://www.basketball-reference.com/players/c/bassbr01/gamelog/2007
3142/11522 2007 Brandon Bass
https://www.basketball-reference.com/players/c/bastoma01/gamelog/2007
3143/11522 2007 Maceo Baston
https://www.basketball-reference.com/players/c/batises01/gamelog/2007
3144/11522 2007 Esteban Batista
https://www.basketball-reference.com/players/c/battito01/gamelog/2007
3145/11522 2007 Tony Battie
https://www.basketball-reference.com/players/c/battish01/gamelog/2007
3146/11522

3219/11522 2007 DeSagana Diop
https://www.basketball-reference.com/players/c/dixonju01/gamelog/2007
3220/11522 2007 Juan Dixon
https://www.basketball-reference.com/players/c/doleami01/gamelog/2007
3221/11522 2007 Michael Doleac
https://www.basketball-reference.com/players/c/doolike01/gamelog/2007
3222/11522 2007 Keyon Dooling
https://www.basketball-reference.com/players/c/doubyqu01/gamelog/2007
3223/11522 2007 Quincy Douby
https://www.basketball-reference.com/players/c/duhonch01/gamelog/2007
3224/11522 2007 Chris Duhon
https://www.basketball-reference.com/players/c/duncati01/gamelog/2007
3225/11522 2007 Tim Duncan
https://www.basketball-reference.com/players/c/dunlemi02/gamelog/2007
3226/11522 2007 Mike Dunleavy
https://www.basketball-reference.com/players/c/duprero01/gamelog/2007
3227/11522 2007 Ronald Dupree
https://www.basketball-reference.com/players/c/ellismo01/gamelog/2007
Error: HTTPSConnectionPool(host='www.basketball-reference.com', port=443): Max retries exceeded with url: /p

UnboundLocalError: local variable 'soup' referenced before assignment

In [42]:
test.to_csv('lines\historical')

In [43]:
test

,Player,G,Date,Age,Tm,,Opp,,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,Tariq Abdul-Wahad,1,1999-11-02,24-364,ORL,@,CHH,L (-14),1,19:34,...,2,4,0,0,0,2,1,4,-0.5,-15
1,Tariq Abdul-Wahad,2,1999-11-03,25-000,ORL,,DET,W (+9),1,28:14,...,6,11,3,1,0,6,2,12,2.6,+7
2,Tariq Abdul-Wahad,3,1999-11-05,25-002,ORL,,CLE,L (-5),1,32:54,...,4,8,0,0,0,2,1,20,13.2,+4
3,Tariq Abdul-Wahad,4,1999-11-06,25-003,ORL,@,WAS,W (+3),1,32:41,...,5,8,2,0,2,1,4,14,13.4,+7
4,Tariq Abdul-Wahad,5,1999-11-08,25-005,ORL,@,HOU,W (+5),1,30:13,...,5,6,1,2,1,0,2,10,8.0,+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,Mike Conley,77,2011-04-02,23-173,MEM,,MIN,W (+17),1,34:15,...,2,3,9,3,0,5,2,5,2.9,+17
77,Mike Conley,78,2011-04-05,23-176,MEM,,LAC,L (-1),1,39:20,...,4,5,4,4,0,1,2,20,18.9,-1
78,Mike Conley,79,2011-04-08,23-179,MEM,,SAC,W (+5),1,39:26,...,5,5,8,2,0,1,2,12,12.4,+16
79,Mike Conley,80,2011-04-10,23-181,MEM,,NOH,W (+22),1,25:29,...,3,3,3,3,0,0,2,2,4.1,+11


In [46]:
test2 = pd.read_csv('lines\historical2',index_col=0)
test2

FileNotFoundError: [Errno 2] No such file or directory: 'lines\\historical2'

In [76]:
# avoid the first header row
rows = soup.findAll('tr')
player_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]
stats = pd.DataFrame(player_stats, columns = headers)


,G,2014 Playoffs,Series,Tm,,Opp,G#,,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,20-29,2,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,30-39,18,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,40+,25,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,0-9,1,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,10-19,15,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,20-29,18,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,30-39,11,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [14]:
stats.dtypes

Player    object
Pos       object
Age       object
Tm        object
G         object
GS        object
MP        object
FG        object
FGA       object
FG%       object
3P        object
3PA       object
3P%       object
2P        object
2PA       object
2P%       object
eFG%      object
FT        object
FTA       object
FT%       object
ORB       object
DRB       object
TRB       object
AST       object
STL       object
BLK       object
TOV       object
PF        object
PTS       object
dtype: object

In [168]:
with pd.option_context('display.max_rows', None,
                       'display.precision', 3,
                       ):
    display(test.sample(10))

,Player,G,Date,Age,Tm,,Opp,,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
33,Arron Afflalo,34,2010-01-03,24-080,DEN,,PHI,L (-3),1,27:15,...,3,6,3,0,0,2,5,17,12.1,+12
33,LaMarcus Aldridge,34,2010-01-05,24-170,POR,,MEM,L (-4),1,41:05,...,4,7,3,0,1,0,3,11,11.5,-8
56,LaMarcus Aldridge,57,2010-02-24,24-220,POR,@,TOR,W (+14),1,39:13,...,2,2,1,1,2,1,2,9,3.7,+7
66,LaMarcus Aldridge,67,2010-03-19,24-243,POR,,WAS,W (+2),1,40:14,...,8,12,1,2,1,2,2,19,14.1,+9
8,LaMarcus Aldridge,9,2009-11-11,24-115,POR,@,MIN,W (+23),1,25:23,...,4,4,2,0,0,0,2,11,8.5,+9
11,LaMarcus Aldridge,12,2009-11-16,24-120,POR,@,ATL,L (-4),1,45:41,...,8,9,2,1,0,0,3,18,13.6,-2
4,LaMarcus Aldridge,5,2009-11-03,24-107,POR,,ATL,L (-6),1,39:57,...,11,14,0,2,0,1,3,20,16.9,-9
40,LaMarcus Aldridge,41,2010-01-22,24-187,POR,@,BOS,L (-3),1,47:17,...,8,10,3,0,0,1,3,19,11.7,+3
30,LaMarcus Aldridge,31,2009-12-25,24-159,POR,,DEN,W (+11),1,45:12,...,12,13,3,1,1,2,2,10,10.3,+10
3,Arron Afflalo,4,2009-11-03,24-019,DEN,@,IND,W (+18),0,22:56,...,3,3,1,0,0,1,0,15,11.0,+16


In [169]:
test['Player'].unique()

array(['Arron Afflalo', 'Alexis Ajinca', 'LaMarcus Aldridge'],
      dtype=object)

In [45]:
save = test.copy(deep=True)
save.tail()

,Player,G,Date,Age,Tm,,Opp,,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
2,Marko_Simonovic,3,2023-02-16,23-124,CHI,,MIL,L (-12),0,1:48,...,0,0,0,0,0,0,1,0,-0.4,+8
3,Marko_Simonovic,4,2023-03-31,23-167,CHI,@,CHO,W (+30),0,3:58,...,0,1,0,0,0,0,1,0,-0.4,-1
4,Marko_Simonovic,5,2023-04-04,23-171,CHI,,ATL,L (-18),0,2:06,...,0,0,0,0,0,0,0,2,1.0,0
5,Marko_Simonovic,6,2023-04-05,23-172,CHI,@,MIL,L (-13),0,0:39,...,0,0,0,0,0,0,0,0,0.0,+3
0,Alondes_Williams,1,2022-12-10,23-174,BRK,@,IND,W (+3),0,5:16,...,1,1,0,0,0,2,1,0,-2.1,-5


In [46]:
test['Date'] = pd.to_datetime(test['Date'])
data = test
for i in range(len(data)):
    data.iloc[i,0] = unidecode(data.iloc[i,0]).replace('_',' ')
    
    

    #remove special characters

In [47]:
data = data.reset_index(drop=True)

In [48]:
data.iloc[65]

Player           Jayson Tatum
G                          66
Date      2023-03-17 00:00:00
Age                    25-014
Tm                        BOS
                            @
Opp                       POR
                      W (+14)
GS                          1
MP                      36:29
FG                          8
FGA                        20
FG%                      .400
3P                          2
3PA                        11
3P%                      .182
FT                         16
FTA                        18
FT%                      .889
ORB                         1
DRB                        11
TRB                        12
AST                         4
STL                         3
BLK                         0
TOV                         5
PF                          2
PTS                        34
GmSc                     26.4
+/-                       +11
Name: 65, dtype: object

In [49]:
data['Player'][2]

'Jayson Tatum'

In [50]:
len(data['Player'].unique())

520

In [51]:
info = pd.read_csv('player_index.csv')
for i in range(len(info)):
    info.iloc[i,1] = unidecode(info.iloc[i,1]).replace('_',' ')
print(data['Player'][2])

p2pos = {player:pos for player,pos in zip(info['Player'].to_list(),info['Pos'].to_list())}
pos = []
for i in range(len(data)):
    pos.append(p2pos[data.iloc[i][0]])
pos

data['Pos'] = pos
data

Jayson Tatum


,Player,G,Date,Age,Tm,,Opp,,GS,MP,...,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,Pos
0,Jayson Tatum,1,2022-10-18,24-229,BOS,,PHI,W (+9),1,38:38,...,12,4,1,1,3,1,35,30.1,+10,PF
1,Jayson Tatum,2,2022-10-21,24-232,BOS,@,MIA,W (+7),1,36:47,...,5,4,1,2,2,1,29,22.7,-1,PF
2,Jayson Tatum,3,2022-10-22,24-233,BOS,@,ORL,W (+6),1,37:39,...,8,1,1,2,1,2,40,34.2,+6,PF
3,Jayson Tatum,4,2022-10-24,24-235,BOS,@,CHI,L (-18),1,35:26,...,8,5,0,0,1,2,26,20.7,-8,PF
4,Jayson Tatum,5,2022-10-28,24-239,BOS,,CLE,L (-9),1,42:11,...,7,4,1,1,4,4,32,24.1,-10,PF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25141,Marko Simonovic,3,2023-02-16,23-124,CHI,,MIL,L (-12),0,1:48,...,0,0,0,0,0,1,0,-0.4,+8,C
25142,Marko Simonovic,4,2023-03-31,23-167,CHI,@,CHO,W (+30),0,3:58,...,1,0,0,0,0,1,0,-0.4,-1,C
25143,Marko Simonovic,5,2023-04-04,23-171,CHI,,ATL,L (-18),0,2:06,...,0,0,0,0,0,0,2,1.0,0,C
25144,Marko Simonovic,6,2023-04-05,23-172,CHI,@,MIL,L (-13),0,0:39,...,0,0,0,0,0,0,0,0.0,+3,C


In [52]:
minutes =  data['MP'].to_list()

for i in range(len(minutes)):
    new = minutes[i].split(':')
    res = float(new[0])+(float(new[1])/60)
    minutes[i] = res
data['MP'] = minutes


In [53]:
data.columns = [['Player', 'G', 'Date', 'Age', 'Tm', 'H/A', 'Opp', 'W/L', 'GS', 'MP', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB',
       'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-', 'Pos']]
home = data['H/A'].values

for i in range(len(home)):
    if home[i] == '@':
        home[i] = 0
    else:
        home[i] = 1
home[0:10]


array([[1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0]], dtype=object)

In [54]:
data['H/A'] = home
data = data.replace('CHO','CHA')
data = data.replace('PHO','PHX')
data= data.replace('BRK','BKN')

WL = data['W/L'].values.astype(str)
margin = []
for i in range(len(WL)):
    input_string = WL[i][0]
    output = re.match(r"^[WL]\s*\(([-]?\d+|\+\d+)\)", input_string).group(1)
    margin.append(output)

margin[0:10]
data['W/L']=margin
data['W/L'] = data['W/L'].astype(int)
data.fillna(5)

,Player,G,Date,Age,Tm,H/A,Opp,W/L,GS,MP,...,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,Pos
0,Jayson Tatum,1,2022-10-18,24-229,BOS,1,PHI,9,1,38.633333,...,12,4,1,1,3,1,35,30.1,+10,PF
1,Jayson Tatum,2,2022-10-21,24-232,BOS,0,MIA,7,1,36.783333,...,5,4,1,2,2,1,29,22.7,-1,PF
2,Jayson Tatum,3,2022-10-22,24-233,BOS,0,ORL,6,1,37.650000,...,8,1,1,2,1,2,40,34.2,+6,PF
3,Jayson Tatum,4,2022-10-24,24-235,BOS,0,CHI,-18,1,35.433333,...,8,5,0,0,1,2,26,20.7,-8,PF
4,Jayson Tatum,5,2022-10-28,24-239,BOS,1,CLE,-9,1,42.183333,...,7,4,1,1,4,4,32,24.1,-10,PF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25141,Marko Simonovic,3,2023-02-16,23-124,CHI,1,MIL,-12,0,1.800000,...,0,0,0,0,0,1,0,-0.4,+8,C
25142,Marko Simonovic,4,2023-03-31,23-167,CHI,0,CHA,30,0,3.966667,...,1,0,0,0,0,1,0,-0.4,-1,C
25143,Marko Simonovic,5,2023-04-04,23-171,CHI,1,ATL,-18,0,2.100000,...,0,0,0,0,0,0,2,1.0,0,C
25144,Marko Simonovic,6,2023-04-05,23-172,CHI,0,MIL,-13,0,0.650000,...,0,0,0,0,0,0,0,0.0,+3,C


In [55]:
data['Date'].max()

Date   2023-04-05
dtype: datetime64[ns]

In [56]:
KM_vals = pd.read_csv('KM_vals.csv',index_col=0)

In [57]:
KM_dict = dict(zip(KM_vals.Player,KM_vals.KM))

In [58]:
data.Player.values[0][0]

'Jayson Tatum'

In [59]:
temp = []
for p in data.Player.values:
    if p[0] in KM_dict:
        temp.append(KM_dict[p[0]])
    else:
        temp.append(15)
data['KM'] = temp

In [ ]:
data[['FG%','3P%','FT%']] = data[['FG%','3P%','FT%']].replace(5.00,0)

In [60]:
data.to_csv('hist_data.csv',index=False)

In [61]:
data

,Player,G,Date,Age,Tm,H/A,Opp,W/L,GS,MP,...,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,Pos,KM
0,Jayson Tatum,1,2022-10-18,24-229,BOS,1,PHI,9,1,38.633333,...,4,1,1,3,1,35,30.1,+10,PF,7
1,Jayson Tatum,2,2022-10-21,24-232,BOS,0,MIA,7,1,36.783333,...,4,1,2,2,1,29,22.7,-1,PF,7
2,Jayson Tatum,3,2022-10-22,24-233,BOS,0,ORL,6,1,37.650000,...,1,1,2,1,2,40,34.2,+6,PF,7
3,Jayson Tatum,4,2022-10-24,24-235,BOS,0,CHI,-18,1,35.433333,...,5,0,0,1,2,26,20.7,-8,PF,7
4,Jayson Tatum,5,2022-10-28,24-239,BOS,1,CLE,-9,1,42.183333,...,4,1,1,4,4,32,24.1,-10,PF,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25141,Marko Simonovic,3,2023-02-16,23-124,CHI,1,MIL,-12,0,1.800000,...,0,0,0,0,1,0,-0.4,+8,C,15
25142,Marko Simonovic,4,2023-03-31,23-167,CHI,0,CHA,30,0,3.966667,...,0,0,0,0,1,0,-0.4,-1,C,15
25143,Marko Simonovic,5,2023-04-04,23-171,CHI,1,ATL,-18,0,2.100000,...,0,0,0,0,0,2,1.0,0,C,15
25144,Marko Simonovic,6,2023-04-05,23-172,CHI,0,MIL,-13,0,0.650000,...,0,0,0,0,0,0,0.0,+3,C,15


In [62]:
data.Date.max()

Date   2023-04-05
dtype: datetime64[ns]